In [1]:
import pandas as pd

data = pd.read_csv("weather_classification_data.csv")
data.head()


,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [2]:
paramets = ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
       'Cloud Cover', 'Atmospheric Pressure', 'UV Index', 'Season',
       'Visibility (km)', 'Location', 'Weather Type']


In [3]:
# Analyze the distribution of the target variable
weather_type_distribution = data['Weather Type'].value_counts()

# Analyze the unique values in the categorical columns
categorical_columns = ['Cloud Cover', 'Season', 'Location']
categorical_analysis = {col: data[col].value_counts() for col in categorical_columns}

weather_type_distribution, categorical_analysis


(Weather Type
 Rainy     3300
 Cloudy    3300
 Sunny     3300
 Snowy     3300
 Name: count, dtype: int64,
 {'Cloud Cover': Cloud Cover
  overcast         6090
  partly cloudy    4560
  clear            2139
  cloudy            411
  Name: count, dtype: int64,
  'Season': Season
  Winter    5610
  Spring    2598
  Autumn    2500
  Summer    2492
  Name: count, dtype: int64,
  'Location': Location
  inland      4816
  mountain    4813
  coastal     3571
  Name: count, dtype: int64})

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical variables
label_encoders = {col: LabelEncoder() for col in categorical_columns + ['Weather Type']}
for col, encoder in label_encoders.items():
    data[col] = encoder.fit_transform(data[col])

# Standardize numerical variables
numerical_columns = ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
                     'Atmospheric Pressure', 'UV Index', 'Visibility (km)']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Split the dataset into features (X) and target (y)
X = data.drop(columns=['Weather Type'])
y = data['Weather Type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((10560, 10), (2640, 10), (10560,), (2640,))

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)
#y_pred_prob = model.predit_proba(X_test)[:,1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred, target_names=label_encoders['Weather Type'].classes_)

# Построение ROC_кривой
#fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
#plt.figure(figsize=(8,6))
#plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.2f}")
#plr.plot([0,1], [0,1], linestyle='--', color ='grey')
#plt.xlabel("False Positive Rate")
#plt.ylabel("True Positive Rate")
#plt.title("ROC Curve")
#plt.legend()
#plt.grid()
#plt.snow()

print(accuracy)
      
print( classification_report_result)


0.9140151515151516
              precision    recall  f1-score   support

      Cloudy       0.87      0.92      0.89       660
       Rainy       0.90      0.91      0.91       660
       Snowy       0.95      0.91      0.93       660
       Sunny       0.94      0.92      0.93       660

    accuracy                           0.91      2640
   macro avg       0.92      0.91      0.91      2640
weighted avg       0.92      0.91      0.91      2640

